In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/houseprice/hprice.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets as widgets
from IPython.display import display

In [3]:
def load_data():
    # First try to find the correct path automatically
    for dirname, _, filenames in os.walk('/kaggle/input'):
        if 'hprice.csv' in filenames:
            correct_path = os.path.join(dirname, 'hprice.csv')
            print(f"Found file at: {correct_path}")
            return pd.read_csv(correct_path)
    
    # If automatic detection fails, try common paths
    common_paths = [
        '/kaggle/input/house-price-dataset/hprice.csv',
        '/kaggle/input/house-prices-advanced-regression-techniques/hprice.csv',
        '/kaggle/input/hprice.csv',
        'hprice.csv'
    ]
    
    for path in common_paths:
        try:
            return pd.read_csv(path)
        except FileNotFoundError:
            continue
    
    raise FileNotFoundError(
        "Could not find hprice.csv. Please ensure: \n"
        "1. The file is uploaded to Kaggle\n"
        "2. Check the correct path in the Kaggle file browser\n"
        "3. The filename is exactly 'hprice.csv' (case sensitive)"
    )

data = load_data()

Found file at: /kaggle/input/houseprice/hprice.csv


In [4]:
def preprocess_data(df):
    # Convert yes/no to 1/0
    binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
    df[binary_cols] = df[binary_cols].apply(lambda x: x.map({'yes': 1, 'no': 0}))
    
    # One-hot encode furnishingstatus
    df = pd.get_dummies(df, columns=['furnishingstatus'], drop_first=True)
    
    return df
    

In [5]:
processed_data = preprocess_data(data.copy())  # Add this line before your split

# Now this will work
X = processed_data.drop('price', axis=1)
y = processed_data['price']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
def train_models():
    # Linear Regression
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    # Decision Tree
    dt = DecisionTreeRegressor(random_state=42)
    dt.fit(X_train, y_train)
    
    return lr, dt

lr_model, dt_model = train_models()

In [8]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return rmse, r2

lr_rmse, lr_r2 = evaluate_model(lr_model, X_test, y_test)
dt_rmse, dt_r2 = evaluate_model(dt_model, X_test, y_test)

In [9]:
print("🏠 House Price Prediction")
model_choice = widgets.Dropdown(
    options=['Linear Regression', 'Decision Tree'],
    value='Linear Regression',
    description='Model:'
)

🏠 House Price Prediction


In [10]:
area = widgets.IntSlider(min=1000, max=20000, value=5000, description='Area (sq ft):')
bedrooms = widgets.IntSlider(min=1, max=6, value=3, description='Bedrooms:')
bathrooms = widgets.IntSlider(min=1, max=4, value=2, description='Bathrooms:')
stories = widgets.IntSlider(min=1, max=4, value=2, description='Stories:')
mainroad = widgets.RadioButtons(options=['Yes', 'No'], description='Main Road:')
guestroom = widgets.RadioButtons(options=['Yes', 'No'], description='Guest Room:')
basement = widgets.RadioButtons(options=['Yes', 'No'], description='Basement:')
hotwaterheating = widgets.RadioButtons(options=['Yes', 'No'], description='Hot Water:')
airconditioning = widgets.RadioButtons(options=['Yes', 'No'], description='AC:')
parking = widgets.IntSlider(min=0, max=3, value=1, description='Parking:')
prefarea = widgets.RadioButtons(options=['Yes', 'No'], description='Pref Area:')
furnishingstatus = widgets.Dropdown(
    options=['Semi-Furnished', 'Unfurnished', 'Furnished'],
    value='Semi-Furnished',
    description='Furnishing:'
)

In [11]:
display(model_choice)
display(area)
display(bedrooms)
display(bathrooms)
display(stories)
display(mainroad)
display(guestroom)
display(basement)
display(hotwaterheating)
display(airconditioning)
display(parking)
display(prefarea)
display(furnishingstatus)

Dropdown(description='Model:', options=('Linear Regression', 'Decision Tree'), value='Linear Regression')

IntSlider(value=5000, description='Area (sq ft):', max=20000, min=1000)

IntSlider(value=3, description='Bedrooms:', max=6, min=1)

IntSlider(value=2, description='Bathrooms:', max=4, min=1)

IntSlider(value=2, description='Stories:', max=4, min=1)

...

RadioButtons(description='Hot Water:', options=('Yes', 'No'), value='Yes')

RadioButtons(description='AC:', options=('Yes', 'No'), value='Yes')

IntSlider(value=1, description='Parking:', max=3)

RadioButtons(description='Pref Area:', options=('Yes', 'No'), value='Yes')

Dropdown(description='Furnishing:', options=('Semi-Furnished', 'Unfurnished', 'Furnished'), value='Semi-Furnis…